In [56]:
#add necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import os
from sklearn.mixture import GaussianMixture 
import datetime
import operator

In [57]:
RecordWritingPath = '/Users/hemingyi/Documents/capstone/post/result/'
TransportationDataPath = '/Users/hemingyi/Documents/capstone/post/transportation/'
EventDataPath = '/Users/hemingyi/Documents/capstone/post/events/'
comboPath = '/Users/hemingyi/Documents/capstone/combo/'
PostData = '/Users/hemingyi/Documents/capstone/post/'
# dataFile = TransportationDataPath+city+'EdgeYearwiseAggregated.csv'

In [58]:
def readTransDF(TransportationDataPath,city):
    df = pd.read_csv(TransportationDataPath+city+'EdgeYearwiseAggregated.csv')
    return df
def makeGraphfromDf(df):
    G=nx.DiGraph()
    nx.set_edge_attributes(G,'weight', 0)
    for k in df.index:
        G.add_edge(df['start_id'][k],df['end_id'][k],weight=df['amount'][k])
#     nx.write_edgelist(G, comboPath+'temp/%s.net'%city)
    return G

In [59]:
def getComboPartition(G,comboPath,city):
    #save network in net format
    nodenum={}
#     G = makeGraphfromDf(df)
    #create a dictionary transforming nodes to unique numbers
    nodes = list(G.nodes())
#     print('nodes amount: ',len(nodes))
    for i,j in enumerate(list(G.nodes())):
        nodenum[str(j)] = str(i)
#         nodes[str(i)] = str(j)
#     i=0
#     for n in list(G.nodes()):
#         nodenum[n]=i
#         nodes[i]=n
#         i+=1

    tempNetFile = comboPath+'temp/%s.net'%city
    f = open(tempNetFile, 'w')
    f.write('*Arcs\n')

    for e in G.edges(data=True):
        f.write('{0} {1} {2}\n'.format(nodenum[str(e[0])],nodenum[str(e[1])],e[2]['weight']))
    f.close()

    #run combo
    command= comboPath+'/comboCPP '+tempNetFile#+' '+str(maxcom)
    os.system(command)

    #read resulting partition
    partitionFile = comboPath+'temp/'+city + '_comm_comboC++.txt'
    f = open(partitionFile, 'r')
    i=0
    partition={}
    for line in f:
        partition[str(nodes[i])]=str(int(line))
        i+=1
#         print(i)
    f.close()
    os.remove(partitionFile) 
    os.remove(tempNetFile)
#     print("Communities: ",len(set(partition.values())))
    return partition

In [68]:
def getSubCommunity(df,maxComm=None):
    data2 = df[df.start_community == df.end_community]
    communities = list(set(df.start_community))
#     print('communities numbers: ',len(communities))
    SubPartition = {}
    for c in communities:
#         print('detecting sub-communities in community ',c)
        d = data2[data2.start_community == c]
        graph = makeGraphfromDf(d)
        p = getComboPartition(graph,comboPath,city)
        SubPartition[c] = p
    df['start_community'] = df.apply(lambda x: str(x['start_community'])+'.'+str(SubPartition[x['start_community']][str(x['start_id'])]),axis=1)
    df['end_community'] = df.apply(lambda x: str(x['end_community'])+'.'+str(SubPartition[x['end_community']][str(x['end_id'])]),axis=1)
    communityNum = len(df['start_community'].unique())
#     print('communityNum: ', communityNum)
    if maxComm:
        if communityNum >= maxComm:
            return df
        else:
#             print('Continue commuity detection')
            return(getSubCommunity(df))
    else:
        if communityNum >= 10:
            return df
        else:
#             print('Continue commuity detection')
            return(getSubCommunity(df))


In [69]:
def aggregateByCommunities(commdata,city,TransportationDataPath):
    commdata = pd.melt(commdata, id_vars=['start_id'], value_vars=['start_community']).drop_duplicates()[['start_id','value']]
    data = pd.read_csv(TransportationDataPath+city+'EdgeDatewiseAggregated.csv')

    print('Raw shape: ',data.shape)
    if 'Date' in data.columns:
        data['date'] = pd.to_datetime(data.Date)
    elif 'date' in data.columns:
        data['date'] = pd.to_datetime(data.date)
    else:
        print(data.columns)
    data = data.merge(commdata, right_on='start_id', left_on='start_id')
    data = data.merge(commdata, left_on='end_id',right_on='start_id' )
    
    communityData = data[['value_x','value_y','date','amount']]
    communityData = communityData.groupby(['value_x','value_y','date']).sum().reset_index()
    communityData.columns = ['start_id', 'end_id', 'date', 'amount']

    return communityData

In [72]:
# only get sub comm for NYC
def subComm(city,maxComm=None):
#     f = open(RecordWritingPath+'F1ScoreSubCommunity.txt', 'a+')
    print('Initialize')
    df = readTransDF(TransportationDataPath,city)
    print('Community Detection, aim at max', maxComm,' communities')
    G = makeGraphfromDf(df)
#     print(city)
    partition = getComboPartition(G,comboPath,city)
    df['start_community'] = df['start_id'].apply(lambda x: partition[str(x)])
    df['end_community'] = df['end_id'].apply(lambda x: partition[str(x)])
    Commdata = getSubCommunity(df,maxComm)
    print('aggregate date wise data by communities')
    data = aggregateByCommunities(Commdata,city,TransportationDataPath)
    print('Save Aggregated DF to csv')
    data.to_csv(TransportationDataPath+'/SubComm/'+city+'DateWiseSubComm.csv',index=False)
    

In [73]:
subComm('DC')

Initialize
Community Detection, aim at max None  communities
aggregate date wise data by communities
Raw shape:  (639177, 4)
Save Aggregated DF to csv


In [74]:
subComm('Taipei')

Initialize
Community Detection, aim at max None  communities
aggregate date wise data by communities
Raw shape:  (7374816, 4)
Save Aggregated DF to csv


In [75]:
subComm('Chicago')

Initialize
Community Detection, aim at max None  communities
aggregate date wise data by communities
Raw shape:  (145897, 4)
Save Aggregated DF to csv


In [76]:
subComm('NewYork')

Initialize
Community Detection, aim at max None  communities
aggregate date wise data by communities
Raw shape:  (19415921, 4)
Save Aggregated DF to csv
